In [ ]:
import dash
from dash import dcc, html, Input, Output
import dash_bootstrap_components as dbc
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import yfinance as yf
import pandas as pd

app = dash.Dash(__name__, external_stylesheets=[dbc.themes.DARKLY])

simbolo = "ES=F"
futuro = yf.Ticker(simbolo)
datos_historicos = futuro.history(start='2020-01-01', end='2024-01-01')
datos_historicos.index = pd.to_datetime(datos_historicos.index)

fecha_inicio_pandemia = "2020-03-01"
fecha_fin_pandemia = "2021-03-01"

candlestick_fig = go.Figure(data=[go.Candlestick(x=datos_historicos.index,
                                                 open=datos_historicos['Open'],
                                                 high=datos_historicos['High'],
                                                 low=datos_historicos['Low'],
                                                 close=datos_historicos['Close'])])

candlestick_fig.add_shape(type="rect",
                          xref="x",
                          yref="paper",
                          x0=fecha_inicio_pandemia,
                          y0=0,
                          x1=fecha_fin_pandemia,
                          y1=1,
                          fillcolor="LightSalmon",
                          opacity=0.5,
                          layer="below",
                          line_width=0)

# Configurar el diseño de la figura de candlesticks
candlestick_fig.update_layout(
    title="S&P 500",
    xaxis_title="Day",
    yaxis_title="E-mini S&P 500",
    xaxis_rangeslider_visible=False,
    template="plotly_dark"
)

# Price and Volume Chart
price_volume_fig = make_subplots(specs=[[{"secondary_y": True}]])
# Usaremos un azul suave para el volumen y un turquesa claro para el precio de cierre
price_volume_fig.add_trace(go.Bar(x=datos_historicos.index, y=datos_historicos['Volume'], name='Volume', marker_color='rgba(0, 255, 220, 1.4)'), secondary_y=False)
price_volume_fig.add_trace(go.Scatter(x=datos_historicos.index, y=datos_historicos['Close'], mode='lines', name='Close Price', line=dict(color='rgba(0, 255, 222, 0.9)')), secondary_y=True)
price_volume_fig.update_layout(title="Price and Volume", template="plotly_dark")

# Daily Returns
datos_historicos['Retornos'] = datos_historicos['Close'].pct_change()
daily_returns_fig = go.Figure()
# Usaremos el mismo turquesa claro para los retornos diarios, asegurando coherencia visual
daily_returns_fig.add_trace(go.Scatter(x=datos_historicos.index, y=datos_historicos['Retornos'], mode='lines', name='Daily Returns', line=dict(color='rgba(0, 255, 222, 0.9)')))
daily_returns_fig.update_layout(title="Daily Returns", template="plotly_dark")

# Diseño de la aplicación Dash
app.layout = dbc.Container([
    dbc.Tabs([
        dbc.Tab(label="Introducción", tab_id="tab-intro"),
        dbc.Tab(label="Análisis de Futuros", tab_id="tab-eda"),
    ], id="tabs", active_tab="tab-intro"),
    html.Div(id="content")
])

@app.callback(Output("content", "children"), [Input("tabs", "active_tab")])
def switch_tab(at):
    if at == "tab-eda":
        return html.Div([
            dcc.Graph(figure=candlestick_fig),
            dcc.Graph(figure=price_volume_fig),
            dcc.Graph(figure=daily_returns_fig)
        ])
    elif at == "tab-intro":
        return html.P("Esta es una introducción a tu problema...")

if __name__ == '__main__':
    app.run_server()

In [ ]:
import dash
from dash import dcc, html, Input, Output
import dash_bootstrap_components as dbc
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import yfinance as yf
import pandas as pd

app = dash.Dash(__name__, external_stylesheets=[dbc.themes.DARKLY])

# Descargar datos
simbolo = "ES=F"
futuro = yf.Ticker(simbolo)
datos_historicos = futuro.history(start='2020-01-01', end='2024-01-01')
datos_historicos.index = pd.to_datetime(datos_historicos.index)

signals = pd.Series([0] * len(datos_historicos.index), index=datos_historicos.index)

signals.iloc[100] = 1
signals.iloc[200] = -1

# Crear una figura con candlesticks
candlestick_fig = go.Figure(data=[go.Candlestick(x=datos_historicos.index,
                                                 open=datos_historicos['Open'],
                                                 high=datos_historicos['High'],
                                                 low=datos_historicos['Low'],
                                                 close=datos_historicos['Close'])])

# Agregar señales de compra y venta al gráfico de candlestick
buy_signals = datos_historicos.loc[signals == 1]
sell_signals = datos_historicos.loc[signals == -1]

candlestick_fig.add_trace(go.Scatter(x=buy_signals.index, y=buy_signals['High'], mode='markers', name='Buy Signal', marker=dict(color='green', size=10)))
candlestick_fig.add_trace(go.Scatter(x=sell_signals.index, y=sell_signals['Low'], mode='markers', name='Sell Signal', marker=dict(color='red', size=10)))

candlestick_fig.update_layout(
    title="S&P 500",
    xaxis_title="Day",
    yaxis_title="E-mini S&P 500",
    font=dict(
        family="Courier New, monospace",
        size=12,
        color="white"
    ),
    xaxis_rangeslider_visible=False,
    template="plotly_dark"
)

app.layout = dbc.Container([
    dbc.Tabs([
        dbc.Tab(label="Introducción", tab_id="tab-intro"),
        dbc.Tab(label="Análisis de Futuros", tab_id="tab-eda"),
    ], id="tabs", active_tab="tab-intro"),
    html.Div(id="content")
])

@app.callback(Output("content", "children"), [Input("tabs", "active_tab")])
def switch_tab(at):
    if at == "tab-eda":
        return html.Div([
            dcc.Graph(figure=candlestick_fig)
        ])
    elif at == "tab-intro":
        return html.P("Esta es una introducción a tu problema...")

if __name__ == '__main__':
    app.run_server()
